<a href="https://colab.research.google.com/github/Feiquanl/CS5100Project/blob/main/accuracyAfterReasonableSentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis
- to extract subjective information and rate the sentiment according to review Text and compare to orignoal overall score to find out accuracy



In [ ]:
import os, json, gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import string
import nltk

from nltk.corpus import wordnet, stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
df = pd.read_csv('reasonable_reviews100verified.csv')
# Check the population size (number of rows) in your DataFrame
print("Population size:", len(df))


Population size: 60559


<ipython-input-47-96f24e2edd7f>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('reasonable_reviews100verified.csv')


We only forcus on overall and reviewText column, and we can see there is imbalanced data from overall 1 to overall 5. So we need to sort out an equal number of obervation for each class.

In [ ]:
# drop any rows w/ missing values
df = df.dropna(subset=["reviewText"])
print("Population size:", len(df))
#discover the actual counts
print(df.overall.value_counts())

print("Population size:", len(df))

# set sample size to labels w/ minimum count
sample_size =7192
# Collect samples for each class in a list
#samples_list = [df[df.overall == label].sample(n=sample_size, random_state=1) for label in df.overall.unique()]
# Specify the labels you're interested in
target_labels = [4, 5]

# Collect samples for each targeted class in a list
samples_list = [df[df.overall == label].sample(n=sample_size, random_state=1) for label in target_labels]

# Concatenate all the samples into one DataFrame
df_equal_overall = pd.concat(samples_list, ignore_index=True)

Population size: 60559
overall
5.0    48021
4.0     7192
3.0     2834
2.0     1299
1.0     1213
Name: count, dtype: int64
Population size: 60559


Fundamental Preprocess for NLP : lowercasing, punctuations removal, and removal of stopwords.

In [ ]:
stopwords_list = stopwords.words('english')

def ReviewProcessing(df):
  # remove non alphanumeric
  df['review_cleaned'] = df.reviewText.str.replace('[^a-zA-Z0-9 ]', '')
  # lowercase
  df.review_cleaned = df.review_cleaned.str.lower()
  # split into list
  df.review_cleaned = df.review_cleaned.str.split(' ')
  # remove stopwords
  df.review_cleaned = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
  return df

Lemmatiazation for NLP: the process of grouping inflected words into a root word.

In [ ]:
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
def get_lemmatize(sent):
  return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])


In [ ]:
clean_data = ReviewProcessing(df_equal_overall)
clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)
clean_data['review_cleaned_lemmatized'] = clean_data.review_cleaned.apply(get_lemmatize)

Implementing TF-IDF Weighting : to measure how important or relevant a term is within the sentence

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

Stochastic Gradient Descent: SGD algorithm computes the minimum of the coust function

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])

Logistic Regression Classifier: categorizing dazta into binary or multiplr groups with discrete prediction value.

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

Split dataset into train and test sets

In [ ]:
x = clean_data['review_cleaned_lemmatized']
y = clean_data['overall']
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2, stratify=y, random_state = 44)


Return the accuracy socre, confusion matrix and classification repost for each report

In [ ]:
# Naive Bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print(accuracy_score(y_test, y_pred_nb))
#print(confusion_matrix(y_test, y_pred_nb))
#print(classification_report(y_test, y_pred_nb))

# SGD Classifier
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print(accuracy_score(y_test, y_pred_sgd))
#print(confusion_matrix(y_test, y_pred_sgd))
#print(classification_report(y_test, y_pred_sgd))

# Logistic Regression
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)
print(accuracy_score(y_test, y_pred_log))
#print(confusion_matrix(y_test, y_pred_log))
#print(classification_report(y_test, y_pred_log))

0.7302745915884602
0.7170663885992353
0.7222801529370872


In [ ]:

from sklearn.model_selection import GridSearchCV

grid=[{'clf__solver': ['lbfgs', 'sag', 'saga'],
       'clf__C': [0.01, 0.1, 1]}]
lr = GridSearchCV(logreg, param_grid = grid, cv = 5, scoring='accuracy', verbose = 1, n_jobs = -1)
best_model = lr.fit(X_train, y_train)

print(best_model.best_estimator_)
print(best_model.best_score_)

y_pred_grid = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_grid))
#print(classification_report(y_test, y_pred_grid))
#print(accuracy_score(y_test, y_pred_grid))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1, max_iter=500))])
0.7015876029215983
[[409 158]
 [192 375]]
